In [1]:
def importAsDF(files, de):
    """
    This Method imports a filelist of txt documents as dataframe
    @param files: a list of files in txt format
    @param de: boolean, if set True the language of the files is set german otherwise english
    @return: dataframe that enlists the txtdocuments content sorted by job, company, job description, language
    """
    startdfs = [0 for x in range(len(files))]

    #import txt files as dataframes
    for i in range(len(files)):
        df= pd.read_csv(files[i], delimiter = "\t", header = None)
        startdfs[i]=df
    #create a more sorted df
    lang_dict = {}
    job_dict = {}
    company_dict = {}
    descr_dict = {}

    for j in range(len(startdfs)):
        #current df
        df = startdfs[j]
        #set language
        if de:
            lang ="de"
        else:
            lang= "en"
        lang_dict[j]= lang;
        #set job
        job = df.iloc[0,0]
        job_dict[j]= job
        #set company
        company = df.iloc[1,0]
        company_dict[j]= company;
        #set description - rest of the dataframe
        descr = ""
        for i in range(2,len(df.index)):
            descr = descr + " "+ df.iloc[i,0].strip("◾")
        descr_dict[j] = descr
    #convert dicts to series
    lang_series = pd.Series(lang_dict)
    job_series = pd.Series(job_dict)
    company_series = pd.Series(company_dict)
    descr_series = pd.Series(descr_dict)
    #convert series to df
    df = pd.DataFrame({'job':job_series, 'company':company_series, 'description':descr_series, 'language':lang_series,})
    return df

In [2]:
def getjobdescriptions(df):
    """
    This Method puts all jobdescriptions of a df into one string
    @param df: The Job dataframe from which to take the descriptions from
    @return: string filled with all job descriptions at once
    """
    #get all descriptions
    descrstr = ""
    for i in range(len(df)):
        descrstr =descrstr +" "+ df.iloc[i,2]
    return (removespecials(descrstr))

In [3]:

def getjobdescriptionsaslist(df):
    """
    This Method puts all jobdescriptions of a df into one list of strings
    @param df: The Job dataframe from which to take the descriptions from
    @return: list of strings filled with all job descriptions doc by doc
    """
    #get all descriptions
    descrstr = []
    for i in range(len(df)):
        newstr = removespecials(df.iloc[i,2].lower())
        descrstr += [newstr]
    return descrstr

In [4]:

def removespecials(str):
    """
    This Method removes specialcaracters
    @param str: the string from whom those specialcharacters shall be removed
    @return: string without those specialcaracters
    """
    return str.replace("◾", "").replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace("•", "")

In [5]:

def firstspacy(descrstr, de):
    """
    This Method takes a string and returns a dataframe with counted occurence of propernouns. It detects compound nouns and counts them as those compound nouns but not as single nouns
    @param str: the string win which to found nouns
    @param de:  boolean, if set True the language of the files is set german otherwise english
    @return: dataframe of single proper nouns and compound proper nouns and their count
    """
    #let the spacy magic begin ^^
    import spacy
    from spacy.tokens import Doc
    from spacy.lang.de import German
    from spacy.lang.en import English
    
    if(de):
        nlp = spacy.load("de_core_news_sm")
    else:
        nlp = spacy.load("en_core_web_sm") 
    
    nlp.max_length = 2000000
    
    nlp.add_pipe("merge_entities")
    
    doc = nlp(descrstr)

    #Matching
    from spacy.matcher import Matcher

    #get compound proper nouns
    matcher = Matcher(nlp.vocab)
    pattern = [{"POS": "PROPN"}, {"POS": "PROPN"}]
    pattern2 = [{"POS": "PROPN"}]
    matcher.add("pat", [pattern])
    matcher.add("pat", [pattern2])
    matches = matcher(doc)

    results =  [doc[start:end].text for match_id, start, end in matches]
    
    #put compound proper nouns nouns in dict and count them
    pnounscount = {}
    pnouns = {}
    index = []
    for result in results:
            if result in pnounscount:
                pnounscount[result] = pnounscount[result]+1
            else:
                pnounscount[result] = 1
                pnouns[result] = result
                index +=[result]
    
    #convert to series and dataframe, sort by count column descending
    pncount = pd.Series(pnounscount)
    pn = pd.Series(pnouns)
    df = pd.DataFrame({'count':pncount, 'term':pn}, index = index)
    df = df.sort_values(by = 'count', ascending = False)
    
    """
    #Is skill Erweiterung per whitelist - TODO: Ausbauen
    from spacy.tokens import Token

    is_skill = lambda token: token.text in ['Java', 'C#', 'Python','C']
    Token.set_extension('skill', getter = is_skill, force = True)
    doc = nlp('Java, C# and Python are programming languages')

    print ([(token.text , token._.skill) for token in doc])
    """
    
    return df

In [6]:

def plot(df):
    """
    This Method plots a dataframe of proper nouns and their counts in a bar diagram
    @param df: dataframe to plot
    """ 
    #Plot 
    import matplotlib.pyplot as plt
    plt.figure()
    plt.ylabel('Anzahl der Nennungen')
    plt.xlabel('Begriff')
    df['count'].plot(kind='bar')

In [7]:
def getDF(files, de):
    """
    This Method uses other methods to take a list of txt files and convert it into a dataframe with proper nouns and their count within those strings
    @param files: a filelist of txt files
    @param de: boolean, true -> german , false -> english
    @return: dataframe that contains propernouns and their count
    """
    df1 = importAsDF(files,de)
    df1.to_csv(r"C:\Users\thegy\Documents\Studium\HS FL 4\Mining\DFs\jobdf.csv", sep=";",decimal=",", index=False)
    descr = getjobdescriptions(df1)
    #descr String nicht lower, replace 3 hauptjob in Beschreibungsstrings
    df = firstspacy(descr, de)
    return df

In [8]:
def dropstopwords(df):
    
    #get all companys
    df11 = importAsDF(files,True)
    comp = ""
    for i in range(len(df11)):    
        comp = comp +" "+ (df11.iloc[i,1])
        
    stopwords = comp.split(" ")
    stopwords += ['d','Data Engineer','Data Analyst','Data Scientist', 'and', '€', 'EUR', 'English','Frankfurt','Main','UK']
    
    for i in range(len(stopwords)):
        if stopwords[i] in df.index:
            df = df.drop(stopwords[i])
    return df

In [ ]:
"""
Read txt files, count proper nouns (compound proper nouns as well and those compounds explicitly not as single nouns)
Get Dataframe of those nouns and ther count and plot them
"""

#find files
import pandas as pd
import glob

files = glob.glob(r"C:\Users\thegy\Documents\Studium\HS FL 4\Mining\Jobs DE\*.txt")
df = getDF(files, True)

filesen = glob.glob(r"C:\Users\thegy\Documents\Studium\HS FL 4\Mining\Jobs EN\*.txt")
dfen = getDF(filesen, False)

In [ ]:
df = dropstopwords(df)
dfen = dropstopwords(dfen)

In [ ]:
print('Filecount de: ',len(files))
print('Filecount en: ',len(filesen))

In [ ]:
plot(df[:20])
plot(dfen[:20])

In [ ]:
df.to_csv(r"C:\Users\thegy\Documents\Studium\HS FL 4\Mining\DFs\nouns_de_retok.csv", sep=";",decimal=",", index=False)
dfen.to_csv(r"C:\Users\thegy\Documents\Studium\HS FL 4\Mining\DFs\nouns_en_retok.csv", sep=";",decimal=",",index=False)